# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 21, 7, 0

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = True

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
              'STOCH': {'fastk_period': [5, 7, 9, 11], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [3, 5, 7, 9], 'low_bound': [10, 15, 20, 25]}}
# optim_dict = {'RSI': {'timeperiod': [16], 'low_bound': [35]},
#                   'STOCH': {'fastk_period': [5], 'slowk_period': [4],
#                             'slowd_period': [5], 'low_bound': [10]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 3840


  0%|                                                                                                                                                                                                 | 0/3840 [00:00<?, ?it/s]


Load the datasets...
Binance
BinanceFutures


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3840/3840 [1:57:16<00:00,  1.83s/it]


,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
2227,STOCH_RSI,14,30,9,5,3,25,84.85,83.84,77.78,76.77,80.81,78.79,77.78,80.81,78.79,78.79,76.77,70.71,69.70,71.72,74.75,79.80,75.76,77.78,77.78,79.80,79.80,79.80,81.82,83.84,0.23,0.27,0.30,0.37,0.46,0.55,0.50,0.51,0.56,0.64,0.56,0.55,0.56,0.58,0.61,0.66,0.66,0.67,0.68,0.72,0.79,0.79,0.92,1.10,99,78.285000,820.215000
3763,STOCH_RSI,16,35,9,5,3,25,80.50,78.50,73.50,73.00,72.50,72.50,72.50,75.50,75.50,76.00,73.00,70.00,68.50,70.00,72.00,75.50,72.50,75.00,73.50,76.00,74.00,74.00,76.50,77.00,0.19,0.25,0.29,0.30,0.36,0.43,0.37,0.45,0.44,0.45,0.47,0.50,0.46,0.44,0.55,0.55,0.58,0.60,0.60,0.64,0.70,0.75,0.90,0.96,200,74.062500,812.500000
2146,STOCH_RSI,14,30,7,4,3,20,85.88,83.53,74.12,76.47,78.82,82.35,81.18,81.18,76.47,77.65,78.82,77.65,75.29,74.12,78.82,80.00,80.00,77.65,78.82,83.53,84.71,82.35,78.82,78.82,0.16,0.21,0.25,0.30,0.41,0.47,0.48,0.49,0.56,0.59,0.56,0.52,0.51,0.58,0.63,0.70,0.68,0.73,0.60,0.71,0.78,0.78,0.72,0.86,85,79.460417,804.135417
2163,STOCH_RSI,14,30,7,5,3,25,85.86,81.82,75.76,75.76,79.80,78.79,79.80,79.80,78.79,75.76,74.75,70.71,70.71,70.71,76.77,78.79,75.76,76.77,78.79,81.82,80.81,80.81,81.82,82.83,0.23,0.27,0.29,0.37,0.46,0.55,0.50,0.41,0.54,0.59,0.56,0.53,0.55,0.58,0.63,0.75,0.64,0.73,0.60,0.77,0.76,0.78,0.85,1.05,99,78.074583,799.383750
2099,STOCH_RSI,14,30,5,5,3,25,85.88,84.71,76.47,76.47,80.00,82.35,80.00,82.35,78.82,80.00,82.35,75.29,71.76,75.29,78.82,78.82,75.29,77.65,80.00,81.18,80.00,80.00,78.82,81.18,0.18,0.27,0.27,0.31,0.42,0.51,0.50,0.39,0.52,0.59,0.56,0.50,0.50,0.60,0.66,0.76,0.68,0.74,0.69,0.74,0.70,0.73,0.78,0.90,85,79.312500,791.562500
2243,STOCH_RSI,14,30,11,2,3,25,80.17,77.59,75.00,75.86,78.45,76.72,72.41,75.86,76.72,75.00,73.28,75.86,76.72,80.17,80.17,78.45,75.00,75.86,75.00,78.45,77.59,78.45,72.41,75.86,0.19,0.28,0.31,0.34,0.43,0.43,0.41,0.49,0.52,0.58,0.57,0.52,0.58,0.62,0.63,0.66,0.72,0.73,0.63,0.70,0.74,0.87,0.91,0.92,116,76.543750,759.075000
2083,STOCH_RSI,14,30,5,4,3,25,84.52,85.71,75.00,80.95,80.95,86.90,82.14,82.14,80.95,79.76,80.95,76.19,75.00,73.81,78.57,76.19,76.19,75.00,75.00,77.38,79.76,79.76,76.19,77.38,0.18,0.24,0.24,0.28,0.41,0.47,0.46,0.48,0.56,0.61,0.60,0.53,0.56,0.61,0.67,0.68,0.76,0.70,0.69,0.74,0.78,0.75,0.80,0.91,84,79.016250,757.365000
2290,STOCH_RSI,14,30,11,5,3,20,82.93,87.80,81.71,78.05,82.93,80.49,76.83,82.93,80.49,79.27,80.49,73.17,71.95,75.61,75.61,81.71,76.83,76.83,78.05,80.49,78.05,79.27,78.05,80.49,0.22,0.28,0.32,0.38,0.47,0.49,0.57,0.43,0.54,0.60,0.56,0.55,0.54,0.62,0.57,0.69,0.58,0.66,0.59,0.74,0.77,0.74,0.89,1.05,82,79.167917,751.769167
2291,STOCH_RSI,14,30,11,5,3,25,83.16,87.37,82.11,77.89,81.05,78.95,76.84,81.05,78.95,77.89,76.84,70.53,70.53,72.63,73.68,80.00,73.68,75.79,76.84,78.95,77.89,77.89,77.89,80.00,0.22,0.28,0.33,0.39,0.47,0.50,0.50,0.43,0.53,0.59,0.54,0.55,0.55,0.55,0.57,0.64,0.57,0.64,0.59,0.74,0.75,0.73,0.88,1.05,95,77.850000,745.750000
3831,STOCH_RSI,16,35,11,5,5,

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [30]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [5],
                            'slowd_period': [3], 'low_bound': [25]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)